## 1. DQN개념
- DQN은 Deep Q-network의 줄임말.
- 강화학습 알고리즘으로 유명한 Q-learning을 딥러닝으로 구현했다는 말

강화학습이란 어떤 환경에서 인공지능 에이전트가 현재상태(환경)을 판단하여 가장 이로운 행동을 하게 만드는 학습 방법이다.
- 이로운행동 -> 보상을 줌
- 해로운행동 -> 패널티를 줌
- 즉 누적된 이득이 최대가 되게 행동하도록 학습이 진행된다.

> Q-learning
: 어떤 상태에서 특정행동을 했을때 가치를 나타내는 함수인 Q함수를 학습하는 알고리즘

이 Q러닝을 신경망으로 구현하면 학습이 상당히 불안해 질 수 있다. 이는 아래 두 방법을 사용하여 해결
1. 먼저 과거의 상태를 기억한 뒤 그중 임의의 상태를 뽑아 학습 
    1. 이렇게하면 특수한 상황에 치우치지않도록 조절할 수 있다.
2. 손실값을 계산하기위해 학습을 진행하면서 최적의 행동을 얻어내는 기본신경망과 얻어낸 값이 좋은 선택인지 비교하는 목표신경망을 분리하는 방법을 사용한다.
    1. 목표신경망은 계속 갱신하는 것이 아니라 기본신경망의 학습된 결괏값을 일정주기마다 목표신경망에 갱신한다.
   
그리고 DQN은 화면의 상태, 즉 화면영상만으로 게임을 학습한다. 따라서 이미지 인식에 뛰어난 CNN을 사용하여 신경망 모델을 구성

## 2. 에이전트 구현하기
- OpenAI라는 비영리회사에서 제공하는 Gym(https://gym.openai.com/) 이라는 강화학습 알고리즘 개발도구가 있음. 
- 책 저자가 만들어놓은 간단한 게임 game.py를 이용

- 에이전트 : 게임의 상태를 입력받아 신경망으로 전달, 신경망에서 판단한 행동을 게임에 적용해서 다음단계로 진행
- 에이전트는 train모드와 게임실행모드(replay)로 나뉜다. 
    - 학습 모드일때는 게임을 화면에 보여주지 않은 채 빠르게 실행하여 학습속도를 높이고
    - 게임실행모드에서는 학습한 결과를 이용하여 게임을 진행하면서 화면에 출력도 해준다.

In [1]:
import tensorflow as tf
import numpy as np
import random
import time
from game import *
from model import DQN

In [2]:
# 에이전트 실행시 모드를 나누어 실행할 수 있도록 tf.app.flags를 이용해 실행시 받을 옵션들을 설정한다.
tf.app.flags.DEFINE_boolean("train",False,"학습모드. 게임을 화면에 보여주지 않습니다.")
FLAGS=tf.app.flags.FLAGS

In [3]:
MAX_EPISODE=10000 # 최대로 학습할 게임 횟수
TARGET_UPDATE_INTERVAL=1000 #학습을 일정횟수만큼 진행할 떄 마다 한번씩 목표신경망을 갱신하라는 옵션
TRAIN_INTERVAL=4 # 게임 4프레임(상태)마다 한번씩 학습하라는 얘기
OBSERVE=100 #일정수준의 학습데이터가 쌓이기 전까지는 학습하지 않고 지켜보기만 하라는 것.

In [4]:
# 해당 게임에서 취할 수 있는 행동은 좌,우,상태유지 이렇게 세개이다(떨어지는 물건을 좌우로 피하는 게임)
# 게임화면은 가로6칸, 세로10칸으로 설정하였다.
NUM_ACTION=3 # 행동 -0:좌,1:유지,2:우
SCREEN_WIDTH=6
SCREEN_HEIGHT=10

In [5]:
def train():
    print('뇌세포 깨우는 중..')
    sess = tf.Session()

    game = Game(SCREEN_WIDTH, SCREEN_HEIGHT, show_game=False)
    brain = DQN(sess, SCREEN_WIDTH, SCREEN_HEIGHT, NUM_ACTION)

    rewards = tf.placeholder(tf.float32, [None])
    tf.summary.scalar('avg.reward/ep.', tf.reduce_mean(rewards))

    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    writer = tf.summary.FileWriter('logs', sess.graph)
    summary_merged = tf.summary.merge_all()

    # 타겟 네트웍을 초기화합니다. # 목표신경망을 한번 초기돠 해준다. 학습된 결과가 없으므로 여기서 목표신경망값은 초기화된 기본 신경망 값과 같다.
    brain.update_target_network()

    # 다음에 취할 액션을 DQN 을 이용해 결정할 시기를 결정합니다.
    epsilon = 1.0
    # 프레임 횟수
    time_step = 0
    total_reward_list = []

    # 게임을 시작합니다.
    for episode in range(MAX_EPISODE):
        terminal = False
        total_reward = 0

        # 게임을 초기화하고 현재 상태를 가져옵니다.
        # 상태는 screen_width x screen_height 크기의 화면 구성입니다.
        state = game.reset()
        brain.init_state(state)

        while not terminal:
            # 입실론이 랜덤값보다 작은 경우에는 랜덤한 액션을 선택하고
            # 그 이상일 경우에는 DQN을 이용해 액션을 선택합니다.
            # 초반엔 학습이 적게 되어 있기 때문입니다.
            # 초반에는 거의 대부분 랜덤값을 사용하다가 점점 줄어들어
            # 나중에는 거의 사용하지 않게됩니다.
            if np.random.rand() < epsilon:
                action = random.randrange(NUM_ACTION)
            else:
                action = brain.get_action()

            # 일정 시간이 지난 뒤 부터 입실론 값을 줄입니다.
            # 초반에는 학습이 전혀 안되어 있기 때문입니다.
            if episode > OBSERVE:
                epsilon -= 1 / 1000

            # 결정한 액션을 이용해 게임을 진행하고, 보상과 게임의 종료 여부를 받아옵니다.
            state, reward, terminal = game.step(action)
            total_reward += reward

            # 현재 상태를 Brain에 기억시킵니다.
            # 기억한 상태를 이용해 학습하고, 다음 상태에서 취할 행동을 결정합니다.
            brain.remember(state, action, reward, terminal)

            if time_step > OBSERVE and time_step % TRAIN_INTERVAL == 0:
                # DQN 으로 학습을 진행합니다.
                brain.train()

            if time_step % TARGET_UPDATE_INTERVAL == 0:
                # 타겟 네트웍을 업데이트 해 줍니다.
                brain.update_target_network()

            time_step += 1

        print('게임횟수: %d 점수: %d' % (episode + 1, total_reward))

        total_reward_list.append(total_reward)

        if episode % 10 == 0:
            summary = sess.run(summary_merged, feed_dict={rewards: total_reward_list})
            writer.add_summary(summary, time_step)
            total_reward_list = []

        if episode % 100 == 0:
            saver.save(sess, 'model/dqn.ckpt', global_step=time_step)
    

In [6]:
def replay():
    print('뇌세포 깨우는 중..')
    sess = tf.Session()

    game = Game(SCREEN_WIDTH, SCREEN_HEIGHT, show_game=True)
    brain = DQN(sess, SCREEN_WIDTH, SCREEN_HEIGHT, NUM_ACTION)

    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('model')
    saver.restore(sess, ckpt.model_checkpoint_path)

    # 게임을 시작합니다.
    for episode in range(MAX_EPISODE):
        terminal = False
        total_reward = 0

        state = game.reset()
        brain.init_state(state)

        while not terminal:
            action = brain.get_action()

            # 결정한 액션을 이용해 게임을 진행하고, 보상과 게임의 종료 여부를 받아옵니다.
            state, reward, terminal = game.step(action)
            total_reward += reward

            brain.remember(state, action, reward, terminal)

            # 게임 진행을 인간이 인지할 수 있는 속도로^^; 보여줍니다.
            time.sleep(0.3)

        print('게임횟수: %d 점수: %d' % (episode + 1, total_reward))


In [7]:
def main(_):
    if FLAGS.train:
        train()
    else:
        replay()

In [ ]:
if __name__ == '__main__':
    tf.app.run()